In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display



def url_to_info(url):


    response = requests.get(url)

    content = response.content
    
    soup = BeautifulSoup(content, 'lxml')

    adress_main = soup.find('ul', class_='short-info-list')
    li_elements = adress_main.find_all('li')
    adress_city = li_elements[0].get_text(strip=True)
    adress_district = li_elements[1].get_text(strip=True)
    adress_neighborhood= li_elements[2].get_text(strip=True)
    price_element = soup.find('p', class_='fz24-text price').get_text(strip=True)

    gross_m2 = soup.find_all('ul', class_='adv-info-list')


    

    data = {'City': adress_city, 'District': adress_district, 'Neighborhood': adress_neighborhood }



    for ul in gross_m2:
        li_elements = ul.find_all('li', {'class': 'spec-item'})
        for li in li_elements:
            try:
                txt = li.find('span', {'class': 'txt'}).get_text(strip=True)
                value = li.find_all('span', {'data-v-60d12482': ''})[1].get_text(strip=True)
                data[txt] = value
            except:
                continue


             
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)
    
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    distance_elements = driver.find_elements(By.CLASS_NAME,"whats-near-card-item")
    for element in distance_elements:

    
    # Extract the inner HTML of the element
        html_content = element.get_attribute('innerHTML')
        soup = BeautifulSoup(html_content, 'html.parser')

    # Find all the div elements with the class "whats-near-card-item__description"
        description_divs = soup.find_all('div', class_='whats-near-card-item__description')
        
    # Find all the div elements with the class "whats-near-card-item__distance"
        distance_divs = soup.find_all('div', class_='whats-near-card-item__distance')

    # Iterate over the description divs and corresponding distance divs
        for description_div, distance_div in zip(description_divs, distance_divs):
            # Extract the name and value
            name = description_div.span.get_text(strip=True)
            value_for_name = distance_div.span.get_text(strip=True)
            name_parts = name.split(" - ")
            name = name_parts[0]
            data[name] = value_for_name
            # Print the name and value
            
    data["Rent"] = price_element


    return data



In [2]:
total_csv = []

In [ ]:


for i in range (35,1000):

    
    
    url = f"https://www.hepsiemlak.com/ankara-satilik?page={i}"
    response = requests.get(url)
    if response.status_code == 200:
        print('Request successful')
    elif response.status_code == 403:
        print(f'Request forbitten with status code {response.status_code}')
        pass
    else:
        print(f'too many request  with status code {response.status_code}')
        pass

    time.sleep(25)

    ## I have put time because website had request limit
    content = response.content

    soup = BeautifulSoup(content, 'lxml')

    value = soup.find_all("div",class_ ="links")

    for value in value:
        link_elements = value('a', class_='card-link')

        links = []
        for link_element in link_elements:
            href = link_element['href']
            full_url = f'https://www.hepsiemlak.com{href}'
            links.append(full_url)

        for link in links:
            
            data = url_to_info(link)

            total_csv.append(data)

    print(f"last scrapped page is {i}")
    import pandas as pd 
    total_data =pd.DataFrame(total_csv)
    total_data.to_csv("house_prices_detailed2.csv")